# 第二阶段：基础实现 (The "How" - Basic)

**目标**：手写一个最简单的 Registry，掌握装饰器（Decorator）的用法，并从配置字典中构建实例。

本 Notebook 将会逐步实现以下功能：
1.  **从 `if-else` 地狱说起**: 展示传统方法的痛点。
2.  **基于字典的朴素实现**: 使用一个简单的全局字典来管理类。
3.  **引入装饰器**: 编写一个 `Registry` 类和 `@register` 装饰器，实现自动注册。
4.  **对象的实例化**: 编写 `build` 方法，根据配置动态创建对象。

## 1. 从 `if-else` 地狱说起

在很多项目中，我们经常需要根据一个字符串（比如从配置文件中读取的）来创建不同的对象实例。一个最直观的写法就是使用 `if-else`。

In [1]:
# 定义几个模型类
class ResNet:
    def __init__(self, depth=50):
        self.depth = depth
        print(f"Creating ResNet with depth {self.depth}")

class VGG:
    def __init__(self, depth=16):
        self.depth = depth
        print(f"Creating VGG with depth {self.depth}")

def create_model(cfg):
    """根据配置字典创建模型"""
    model_type = cfg.get('type')
    if model_type == 'ResNet':
        return ResNet(depth=cfg.get('depth', 50))
    elif model_type == 'VGG':
        return VGG(depth=cfg.get('depth', 16))
    else:
        raise ValueError(f"Unknown model type: {model_type}")

# 使用
cfg1 = {'type': 'ResNet', 'depth': 101}
model1 = create_model(cfg1)

cfg2 = {'type': 'VGG'}
model2 = create_model(cfg2)

Creating ResNet with depth 101
Creating VGG with depth 16


**痛点分析:**
上面的 `create_model` 函数存在一个严重的问题：每当我们新增一个模型（比如 `MobileNet`），就必须去修改 `create_model` 函数的代码，增加一个 `elif` 分支。这违反了软件工程中的**开闭原则**（Open-Closed Principle）：对扩展开放，对修改关闭。

我们需要一种机制，在新增模型时，完全不需要改动创建实例的核心代码。这就是注册机制要解决的问题。

## 2. 基于字典的朴素实现

一个简单的改进是使用字典来维护字符串和类之间的映射关系。

In [5]:
# 1. 维护一个全局的注册表（就是个字典）
MODELS_DICT = {}

# 2. 手动注册模型
# 假设 ResNet 和 VGG 已经在前面定义过了
MODELS_DICT['ResNet'] = ResNet
MODELS_DICT['VGG'] = VGG

# 3. 重写创建函数
def create_model_v2(cfg):
    """从字典中查找并创建模型"""
    # 使用 pop 获取类型，剩下的就是参数。深拷贝一下以防修改原始cfg
    cfg_ = cfg.copy()
    model_type = cfg_.pop('type') 
    model_class = MODELS_DICT.get(model_type)
    if model_class is None:
        raise ValueError(f"Unknown model type: {model_type}")
    
    return model_class(**cfg_) # 使用**kwargs传递参数

# 使用
print("--- Using Dictionary-based factory ---")
cfg3 = {'type': 'ResNet', 'depth': 101}
model3 = create_model_v2(cfg3)

cfg4 = {'type': 'VGG', 'depth': 19}
model4 = create_model_v2(cfg4)

--- Using Dictionary-based factory ---
Creating ResNet with depth 101
Creating VGG with depth 19


In [15]:
print(MODELS_DICT)  # 输出注册表内容

{'ResNet': <class '__main__.ResNet'>, 'VGG': <class '__main__.VGG'>}


这种方法比 `if-else` 好多了！现在如果我们要添加 `MobileNet`，只需要在定义完类之后，手动调用 `MODELS_DICT['MobileNet'] = MobileNet` 即可，`create_model_v2` 函数完全不用动。

**新的痛点:**
虽然核心逻辑解耦了，但我们仍然需要**手动注册**。在大型项目中，模型定义可能分散在不同的文件里，很容易忘记去手动更新那个全局字典。有没有办法让这个注册过程自动化呢？—— 装饰器登场！

## 3. 引入装饰器，实现自动注册

我们可以封装一个 `Registry` 类，它负责管理内部的字典，并提供一个装饰器方法 `@register_module()` 来自动完成注册。

In [23]:
class Registry:
    """一个通用的注册器类"""
    def __init__(self, name):
        self._name = name
        self._module_dict = {}

    def __repr__(self):
        return f"Registry(name={self._name}, modules={list(self._module_dict.keys())})"

    @property
    def name(self):
        return self._name

    @property
    def module_dict(self):
        return self._module_dict

    def register_module(self, name=None, force=False):
        """
        注册一个模块（类）的装饰器。
        
        Args:
            name (str, optional): 注册的模块名。如果为 None，则使用类名。
            force (bool, optional): 是否强制覆盖已存在的模块。
        """
        def decorator(cls):
            module_name = name if name is not None else cls.__name__
            if not force and module_name in self._module_dict:
                raise KeyError(f"{module_name} is already registered in {self.name}")
            self._module_dict[module_name] = cls
            return cls
        
        return decorator

### 使用 Registry 和装饰器

现在，注册模型变得非常简单和优雅。

In [17]:
# 1. 创建一个模型注册表实例
MODELS = Registry('models')

# 2. 使用装饰器自动注册
@MODELS.register_module()
class ResNetV2: # 改个名字避免和之前的冲突
    def __init__(self, depth=50):
        self.depth = depth
        print(f"Creating ResNetV2 with depth {self.depth} (via Registry)")

@MODELS.register_module()
class VGGV2: # 改个名字避免和之前的冲突
    def __init__(self, depth=16):
        self.depth = depth
        print(f"Creating VGGV2 with depth {self.depth} (via Registry)")

# 也可以指定一个不同的名字
@MODELS.register_module(name='MobileNetV2')
class MobileNet:
    def __init__(self, version=2):
        self.version = version
        print(f"Creating MobileNet v{self.version}")


# 查看注册表内容
print(MODELS)
print(MODELS.module_dict)

Registry(name=models, modules=['ResNetV2', 'VGGV2', 'MobileNetV2'])
{'ResNetV2': <class '__main__.ResNetV2'>, 'VGGV2': <class '__main__.VGGV2'>, 'MobileNetV2': <class '__main__.MobileNet'>}


## 4. 对象的实例化 (Build)

最后一步，我们为 `Registry` 类添加一个 `build` 方法，它能根据配置字典（config）来查找注册过的类并创建实例。这使得创建过程和注册过程都封装在了一起。

In [18]:
import copy

# 为了不重复定义，我们这里直接给上面定义的 Registry 类 "打补丁"
# 在实际项目中，build 方法会直接写在类的定义里

def build_from_cfg(self, cfg, **default_args):
    """
    根据配置字典构建一个实例。
    
    Args:
        cfg (dict): 包含 'type' key 的配置字典。
        default_args (dict, optional): 默认参数。
    """
    # 深拷贝一份配置，避免修改原始配置
    cfg = copy.deepcopy(cfg)
    
    # 合并默认参数
    if default_args:
        for name, value in default_args.items():
            cfg.setdefault(name, value)
            
    # 获取要构建的类型
    obj_type = cfg.pop('type')
    
    # 在注册表中查找对应的类
    obj_cls = self._module_dict.get(obj_type)
    if obj_cls is None:
        raise KeyError(f"{obj_type} is not registered in {self.name}")
        
    # 用剩下的参数实例化类
    return obj_cls(**cfg)

# 将 build 方法动态添加到 Registry 类上
Registry.build = build_from_cfg

### 使用 `build` 方法创建实例

现在，整个流程 "注册 -> 配置 -> 创建" 就完整了。

In [19]:
# 1. 定义配置
# 注意这里我们使用新的类名 ResNetV2, VGGV2
cfg_resnet = {'type': 'ResNetV2', 'depth': 101}
cfg_vgg = {'type': 'VGGV2', 'depth': 19}
cfg_mobile = {'type': 'MobileNetV2'}

# 2. 使用 registry.build() 来创建实例
model_resnet = MODELS.build(cfg_resnet)
model_vgg = MODELS.build(cfg_vgg)
model_mobile = MODELS.build(cfg_mobile)

# 3. 检查创建的对象
print(f"Created: {model_resnet}")
print(f"Created: {model_vgg}")
print(f"Created: {model_mobile}")

Creating ResNetV2 with depth 101 (via Registry)
Creating VGGV2 with depth 19 (via Registry)
Creating MobileNet v2
Created: <__main__.ResNetV2 object at 0x000002ED63F10E00>
Created: <__main__.VGGV2 object at 0x000002ED63F12060>
Created: <__main__.MobileNet object at 0x000002ED63F111F0>


## 总结

到此为止，我们已经完成了一个基础但功能完备的注册器！回顾一下它的优点：

- **自动化**: 定义类时使用装饰器即可注册，无需手动维护字典。
- **解耦合**: 创建实例的代码 (`build` 方法) 与类的具体实现完全分离。
- **符合开闭原则**: 新增任何模型类，都不需要修改 `Registry` 类或 `build` 方法。只需要在新的文件里定义类并用装饰器注册即可。
- **配置驱动**: 整个程序的行为可以由外部的配置文件（如 YAML 或 JSON）驱动，极大地提高了灵活性。

在下一阶段，我们将探讨在大型项目中，如何解决跨文件自动发现模块等更工程化的挑战。

# 第三阶段：进阶工程挑战 (The "How" - Advanced)

**目标**：解决实际工程中遇到的复杂问题，如模块加载和依赖管理。

## 1. 跨文件与模块自动发现 (Auto Discovery)

**最常见的大坑**：定义了类，加了装饰器，但注册表中没有？

原因：Python 是解释型语言，未 `import` 的文件不会执行，装饰器就不会运行。

**解决方案**：在 `__init__.py` 中显式导入或者使用 `importlib` 动态扫描文件夹，确保每个模块都被遍历。

In [20]:
import importlib
import pkgutil
from pathlib import Path

def auto_register_package(package_name, registry):
    """遍历包并导入所有模块，触发注册装饰器"""
    pkg = importlib.import_module(package_name)
    pkg_path = Path(pkg.__file__).parent
    for finder, module_name, is_pkg in pkgutil.iter_modules([str(pkg_path)]):
        module_fullname = f"{package_name}.{module_name}"
        importlib.import_module(module_fullname)

# 示例：确保所有子模块被导入，装饰器就会自动生效
# auto_register_package('models.backbones', BACKBONES)

## 2. 层级注册与多注册表

大型项目往往需要管理多个域，例如 `BACKBONES`, `HEADS`, `LOSSES`。
分别维护不同的 `Registry` 实例可以让命名空间清晰，避免冲突。

In [21]:
BACKBONES = Registry('backbones')
HEADS = Registry('heads')
LOSSES = Registry('losses')

@BACKBONES.register_module()
class SimpleBackbone:
    def __init__(self, channels=64):
        self.channels = channels

@HEADS.register_module()
class ClassificationHead:
    def __init__(self, num_classes=1000):
        self.num_classes = num_classes

@LOSSES.register_module()
class SoftmaxLoss:
    def __init__(self):
        pass

backbone_cfg = {'type': 'SimpleBackbone', 'channels': 128}
head_cfg = {'type': 'ClassificationHead', 'num_classes': 10}
loss_cfg = {'type': 'SoftmaxLoss'}

backbone = BACKBONES.build(backbone_cfg)
head = HEADS.build(head_cfg)
loss = LOSSES.build(loss_cfg)

## 3. 参数注入与默认值管理

构建阶段有时需要多个共享参数，`build` 方法的 `default_args` 提供默认值，同时嵌套配置可以组合复杂对象。

In [22]:
PROCESSORS = Registry('processors')

@PROCESSORS.register_module()
class Resize:
    def __init__(self, size=(224, 224)):
        self.size = size

@PROCESSORS.register_module()
class Normalize:
    def __init__(self, mean=0.5, std=0.5):
        self.mean = mean
        self.std = std

PIPELINES = Registry('pipelines')

@PIPELINES.register_module(name='PreprocessPipeline')
class PreprocessPipeline:
    def __init__(self, steps=None, mode='train'):
        self.steps = steps or []
        self.mode = mode

def build_pipeline_steps(step_cfgs):
    return [PROCESSORS.build(cfg) for cfg in step_cfgs]

pipeline_cfg = {
    'type': 'PreprocessPipeline',
    'steps': [
        {'type': 'Resize', 'size': (128, 128)},
        {'type': 'Normalize', 'mean': 0.45}
    ]
}

step_configs = pipeline_cfg.pop('steps')
nested_steps = build_pipeline_steps(step_configs)
pipeline = PIPELINES.build(pipeline_cfg, steps=nested_steps, mode='eval')
print(f"Built pipeline in {pipeline.mode} mode with {len(pipeline.steps)} steps")

Built pipeline in eval mode with 2 steps


## 小结

- 自动发现确保所有模块都能注册，减少手动导入。
- 多个注册表维持语义清晰，对不同领域组件进行分离。
- 默认参数与嵌套配置让构建变得灵活，适用于复杂流水线。

下一阶段将转向工业级源码研读，学习成熟的 Registry 实现。